In [1]:
import pandas as pd
import requests
import json
import numpy as np
import time

# Setting  up

In [2]:
# note that CLIENT_ID refers to 'personal use script' and SECRET_TOKEN to 'token'
auth = requests.auth.HTTPBasicAuth('9wXh5oa4cfW07_eUn5Hu3A', 'm3GnhaEvbM5LGCWX3BMghLCatOLN3g')
# setup our header info, which gives reddit a brief description of our app
headers = {'User-Agent': 'babi_research/0.0.1'}
# here we pass our login method (password), username, and password
data = {'grant_type': 'password',
        'username': 'softyarn',
        'password': '1532679!Qa1',
        'User-Agent':'babi_research/0.0.1',
        'refresh_token': '167791397396-1cjKrPIOvr4ct1EFwMm57hY1vi7baw'
        }

In [3]:
# send our request for an OAuth token
res = requests.post('https://www.reddit.com/api/v1/access_token',
                    auth=auth, data=data, headers=headers)
# convert response to JSON and pull access_token value
res.json()

{'access_token': '167791397396-a3TE9cb-1wOUUNjPFJrUyx0rYHfsjg',
 'token_type': 'bearer',
 'expires_in': 86400,
 'scope': '*'}

In [4]:
# convert response to JSON and pull access_token value
token = res.json()['access_token']
# add authorization to our headers dictionary
headers['Authorization'] = 'bearer {}'.format(token)
# while the token is valid (~2 hours) we just add headers=headers to our requests. End point: GET /api/v1/me
requests.get('https://oauth.reddit.com/api/v1/me', headers=headers).json()

{'is_employee': False,
 'seen_layout_switch': False,
 'has_visited_new_profile': False,
 'pref_no_profanity': True,
 'has_external_account': False,
 'pref_geopopular': '',
 'seen_redesign_modal': False,
 'pref_show_trending': True,
 'subreddit': {'default_set': False,
  'user_is_contributor': False,
  'banner_img': '',
  'restrict_posting': True,
  'user_is_banned': False,
  'free_form_reports': True,
  'community_icon': None,
  'show_media': True,
  'icon_color': '',
  'user_is_muted': None,
  'display_name': 'u_softyarn',
  'header_img': None,
  'title': '',
  'coins': 0,
  'previous_names': [],
  'over_18': False,
  'icon_size': [256, 256],
  'primary_color': '',
  'icon_img': 'https://styles.redditmedia.com/t5_3ef3ey/styles/profileIcon_snoo86ebddf0-4659-47c8-9ad6-b6ca0588943d-headshot-f.png?width=256&amp;height=256&amp;crop=256:256,smart&amp;s=86680ec95737b7ce10db712e2889672a4de8bbf1',
  'description': '',
  'allowed_media_in_comments': [],
  'submit_link_label': '',
  'header_size

# Miscellanous scraping

In [5]:
#Get moderators of feminisms
fds_mod= requests.get('https://oauth.reddit.com/r/femaledatingstrategy/about/moderators', headers=headers).json()

In [6]:
fds_mod_df = pd.DataFrame(fds_mod["data"]["children"])
fds_mod_df

,name,author_flair_text,mod_permissions,date,rel_id,id,author_flair_css_class
0,rainisthelife,Ruthless Strategist,[all],1.551256e+09,rb_175c48g,t2_w4w0qu,None
1,TheMarbleSlab,Ruthless Strategist,[all],1.564346e+09,rb_1fpm08m,t2_22udesmu,None
2,TheOGJammies,Ruthless Strategist,[all],1.566522e+09,rb_1h2vrgy,t2_43ex46gt,None
3,AverageToHot,Ruthless Strategist,[all],1.566538e+09,rb_1h398z4,t2_3u7axe0z,None
4,throwawayy92838383,Ruthless Strategist,[all],1.569776e+09,rb_1j0legb,t2_4f0yk6fe,None
5,Liz_Lemondrop,Ruthless Strategist,[all],1.572169e+09,rb_1kflrcs,t2_4vwzr8nf,None
6,Saferbot,Ruthless Strategist,[all],1.572235e+09,rb_1kh6osb,t2_6ladm,None
7,CoolMelonade,Ruthless Strategist,[all],1.572289e+09,rb_1kia3cy,t2_4w9tmtae,None
8,makingballoons,None,"[posts, access, mail, flair]",1.572364e+09,rb_1kjwe6i,t2_ivbk9wq,None
9,NecessaryCook,Ruthless Strategist,[all],1.572386e+09,rb_1kkgl98,t2_22hgl225,None


In [ ]:
fds_mod_df.iloc[6,]

# Function

In [11]:
def clean_data(source_dir, comment_bool):
    df = pd.read_csv(source_dir)
    df = df[["id","body",'author']]
    if comment_bool == True:
        df['id'] ='t1_' + df['id'].astype(str)
    else:
        df['id'] ='t3_' + df['id'].astype(str)
    nomod = df.loc[(df['author'] != "AutoModerator") & (df['author'] != '[deleted]')]
    nomod["note"] = np.where(nomod["body"] == "[removed]", "removed", np.where(nomod["body"] == "[deleted]", "deleted", "text"))
    return nomod

In [12]:
def clean_df(df, comment_bool):
    df = df[["id","body",'author']]
    if comment_bool == True:
        df['id'] ='t1_' + df['id'].astype(str)
    else:
        df['id'] ='t3_' + df['id'].astype(str)
    nomod = df.loc[(df['author'] != "AutoModerator") & (df['author'] != '[deleted]')]
    nomod["note"] = np.where(nomod["body"] == "[removed]", "removed", np.where(nomod["body"] == "[deleted]", "deleted", "text"))
    return nomod

In [13]:
def get_comments(df, start_from) :
    tic = time.time()
    global res_df
    res_df = []
    i = 0
    if start_from == None:
        N = 0
    else:
        N = start_from
    while i < len(df["id"]):
        j = min(len(df["id"]), i + 40)
        x = ",".join(df["id"][i:j])
        comments = requests.get('https://oauth.reddit.com/api/info',headers=headers,params={'id': x}).json()
        for k in range(j-i):
            comment_df = pd.DataFrame.from_dict(comments.get("data").get("children")[k].get("data"), orient = "index").transpose()
            res_df.append(comment_df)
        i += 40
        if i%15000 == 0:
            res_out = pd.concat(res_df)
            res_out.to_csv('unpop_comments_' + str(i/15000) + '.csv', encoding = 'utf_8_sig')
            res = []
        if i%300 == 0:
            time.sleep(10)
            #now = time.localtime(time.time())
            print('Time elapsed: ', round(time.time() - tic),' secs, i = ',i)
            #print(time.strftime("Time now: %HH:%M:%S",now),f', i = {i}')
    return pd.concat(res_df)

In [ ]:
def get_submissions(df):
    res_df = pd.DataFrame()
    i = 0
    while i < len(df["id"]):
        x = ",".join(df["id"][i:i+20])
        submissions = requests.get('https://oauth.reddit.com/by_id/names',headers=headers,params={'names': x}).json()
        for k in range(20):
            submission_df = pd.DataFrame.from_dict(comments.get("data").get("children")[k].get("data"), orient = "index").transpose()
            res_df.append(submission_df)
        if i%300 == 0:
            time.sleep(5)
        i += 20  
    return res_df  

# Scraping

# r/femaledatingstrategy

In [ ]:
fds = clean_data("E:/gihub-data/redditbots/fds/fds_comments.csv", comment_bool= True)

In [ ]:
fds_res = get_comments(fds)

In [ ]:
fds_res.to_csv('fds_comment_retrieved.csv')

In [ ]:
requests.get('https://oauth.reddit.com/by_id/names',headers=headers,params={'names': 't3_cipdyg'}).json()

In [ ]:
requests.get('https://oauth.reddit.com/user/')

## r/AskReddit

In [ ]:
askreddit = clean_data("E:/gihub-data/redditbots/control-fds/askreddit.csv", comment_bool= True)

In [ ]:
askreddit_res = get_comments(askreddit)

In [ ]:
askreddit_res.to_csv('E:/gihub-data/redditbots/control-fds/askreddit_comment_retrieved.csv')

# PurplePillDebate

In [ ]:
purplepill = clean_data("E:/gihub-data/redditbots/control-fds/purple.csv", comment_bool= True)

In [ ]:
purple_res = get_comments(purplepill)

In [ ]:
purple_res.to_csv('E:/gihub-data/redditbots/control-fds/purple_comment_retrieved.csv')

# r/AmItheAsshole

undone

In [ ]:
AITA = clean_data("E:/gihub-data/redditbots/control-fds/AITA.csv", comment_bool= True)

In [ ]:
AITA_res = get_comments(AITA)

# r/unpopularopinion

undone

In [8]:
unpop_df = pd.read_csv("E:/gihub-data/redditbots/control-fds/unpopularopinion.csv",  skiprows= list(range(1,150600)))

In [9]:
unpop_1 = clean_df(unpop_df, comment_bool=True)

C:\Users\nguye\AppData\Local\Temp\ipykernel_27264\332450768.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['id'] ='t1_' + df['id'].astype(str)
C:\Users\nguye\AppData\Local\Temp\ipykernel_27264\332450768.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nomod["note"] = np.where(nomod["body"] == "[removed]", "removed", np.where(nomod["body"] == "[deleted]", "deleted", "text"))


In [14]:
unpop_res = get_comments(unpop_1, start_from = 150600) #start_from 150600

Time elapsed:  17  secs, i =  600
Time elapsed:  33  secs, i =  1200
Time elapsed:  49  secs, i =  1800
Time elapsed:  65  secs, i =  2400
Time elapsed:  82  secs, i =  3000
Time elapsed:  98  secs, i =  3600
Time elapsed:  114  secs, i =  4200
Time elapsed:  130  secs, i =  4800
Time elapsed:  147  secs, i =  5400
Time elapsed:  163  secs, i =  6000
Time elapsed:  179  secs, i =  6600
Time elapsed:  196  secs, i =  7200
Time elapsed:  213  secs, i =  7800
Time elapsed:  230  secs, i =  8400
Time elapsed:  247  secs, i =  9000
Time elapsed:  263  secs, i =  9600
Time elapsed:  281  secs, i =  10200
Time elapsed:  297  secs, i =  10800
Time elapsed:  314  secs, i =  11400
Time elapsed:  332  secs, i =  12000


IndexError: list index out of range

In [ ]:
unpop_res.to_csv('E:/gihub-data/redditbots/control-fds/unpop_comment_retrieved.csv')

# Gender Critical